###### This notebook is heavily based on the 'sentimnent' notebook in the original CheckList repo

## Imports

In [1]:
import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb

In this notebook, we'll define a test suite to "CheckList" different sentiment models.The idea is to check different linguistic capabilities using tests derived from software engineering principles. 


In [16]:
editor = checklist.editor.Editor()
suite = TestSuite()

## Capability: Vocabulary

### MFT (Minimal Functionality Test)

In [17]:
#Adding relevant movie nouns 
movie_noun = ['movie', 'actor', 'director', 'producer', 'crew', 'script', 'production house', 'series', 'sequel']
editor.add_lexicon('movie_noun', movie_noun)

In [18]:
pos_adj = ['good', 'great', 'excellent', 'amazing', 'extraordinary', 'beautiful', 'fantastic', 'nice', 'incredible', 'exceptional', 'awesome', 'perfect', 'fun', 'happy', 'adorable', 'brilliant', 'exciting', 'sweet', 'wonderful']
neg_adj = ['awful', 'bad', 'horrible', 'weird', 'rough', 'lousy', 'unhappy', 'average', 'difficult', 'poor', 'sad', 'frustrating', 'hard', 'lame', 'nasty', 'annoying', 'boring', 'creepy', 'dreadful', 'ridiculous', 'terrible', 'ugly', 'unpleasant']

pos_verb_present = ['like', 'enjoy', 'appreciate', 'love',  'recommend', 'admire', 'value', 'welcome']
neg_verb_present = ['hate', 'dislike', 'regret',  'abhor', 'dread', 'despise' ]

pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
neg_verb_past = ['hated', 'disliked', 'regretted',  'abhorred', 'dreaded', 'despised']


In [19]:
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )

editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)

editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)

editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)

In [20]:
## Create MFTs & add to test suite
test = MFT(pos_adj, labels=1)
suite.add(test, 'single positive words', 'Vocabulary', '')

test = MFT(neg_adj, labels=0)
suite.add(test, 'single negative words', 'Vocabulary', '')

In [21]:
## words in context

t = editor.template('{it} {movie_noun} {be} {pos_adj}.', it=['The', 'This', 'That'], be=['is', 'was'], labels=1, save=True)
t += editor.template('{it} {be} {a:pos_adj} {movie_noun}.', it=['It', 'This', 'That'], be=['is', 'was'], labels=1, save=True)
t += editor.template('{i} {pos_verb} {the} {movie_noun}.', i=['I', 'We'], the=['this', 'that', 'the'], labels=1, save=True)
t += editor.template('{it} {movie_noun} {be} {neg_adj}.', it=['That', 'This', 'The'], be=['is', 'was'], labels=0, save=True)
t += editor.template('{it} {be} {a:neg_adj} {movie_noun}.', it=['It', 'This', 'That'], be=['is', 'was'], labels=0, save=True)
t += editor.template('{i} {neg_verb} {the} {movie_noun}.', i=['I', 'We'], the=['this', 'that', 'the'], labels=0, save=True)
# equivalent to:
# test = MFT(t.data, labels=t.labels, templates=t.templates)
test = MFT(**t)
suite.add(test, 'Sentiment-laden words in context', 'Vocabulary', 'Use positive and negative verbs and adjectives with movie nouns such as movie, crew, director etc."')

## Capability: Negation

### MFT (Minimal Functionality Test)

In [22]:
t = editor.template('{it} {movie_noun} {nt} {pos_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('{it} {benot} {a:pos_adj} {movie_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} {pos_verb_present} {the} {movie_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
t += editor.template('No one {pos_verb_present}s {the} {movie_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=0, templates=t.templates)
suite.add(test, 'simple negations: negative', 'Negation', 'Very simple negations of positive statements')

In [23]:
t = editor.template('{it} {movie_noun} {nt} {neg_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('{it} {benot} {a:neg_adj} {movie_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} {neg_verb_present} {the} {movie_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
t += editor.template('No one {neg_verb_present}s {the} {movie_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
# expectation: prediction is not 0
is_not_0 = lambda x, pred, *args: pred != 0
#test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)
test = MFT(t.data, labels = 1, templates = t.templates)
suite.add(test, 'simple negations: not negative', 'Negation', 'Very simple negations of negative statements. ')

## Capability: temporal awareness

In [24]:
change = ['but', 'even though', 'although', '']
t = editor.template(['I used to think this director was {neg_adj}, {change} now I think he is {pos_adj}.',
                                 'I think this movie is {pos_adj}, {change} I used to think it was {neg_adj}.',
                                 'In the past I thought this series was {neg_adj}, {change} now I think it is {pos_adj}.',
                                 'I think this airline is {pos_adj}, {change} in the past I thought it was {neg_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=200, save=True, labels=1)

In [25]:

t += editor.template(['I used to think this movie was {pos_adj}, {change} now I think it is {neg_adj}.',
                                 'I think this series is {neg_adj}, {change} I used to think it was {pos_adj}.',
                                 'In the past I thought this sequel was {pos_adj}, {change} now I think it is {neg_adj}.',
                                 'I think this actor is {neg_adj}, {change} in the past I thought he was {pos_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=500, save=True, labels=0)
test = MFT(**t)
description = '''Have two conflicing statements, one about the past and one about the present.
Expect the present to carry the sentiment. Examples:
I used to love this movie, now I hate it -> should be negative
I love this movie, although I used to hate it -> should be positive
'''
suite.add(test, 'conflicting statements about past and present', 'Temporal', description)

### Save 

In [26]:
### Save suite as pkl file
path = './sentiment_suite.pkl'
suite.save(path)